In [19]:
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
from PIL import Image
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [12]:
df_train = pd.read_csv("data/train/_annotations.csv")
df_test = pd.read_csv("data/test/_annotations.csv")

In [13]:
df_train.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,egohands-public-1620914960773_png_jpg.rf.aa184...,640,640,Rock,429,185,562,319
1,egohands-public-1624053434391_png_jpg.rf.aaef5...,640,640,Paper,269,354,544,443
2,egohands-public-1624465902684_png_jpg.rf.aaa09...,640,640,Rock,427,332,551,509
3,Screen-Shot-2022-02-08-at-12-59-24-PM_png.rf.a...,640,640,Rock,80,268,145,395
4,egohands-public-1622127402076_png_jpg.rf.aa897...,640,640,Rock,83,128,296,381


In [15]:
df_train['class'].value_counts()

class
Rock        1924
Paper       1349
Scissors    1337
Name: count, dtype: int64

In [14]:
df_test['class'].value_counts()

class
Paper       72
Scissors    67
Rock        65
Name: count, dtype: int64

In [17]:
def get_img(img_src, is_train = True):
    if is_train:
        img_src = f'data/train/{img_src}'
    else:
        img_src = f'data/test/{img_src}'

    img = cv2.imread(img_src)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    return img

In [24]:
def preprocess(df, resize, is_train = True):
    
    processed_data = []

    for index, row in df.iterrows():

        img_src = row['filename']
        img_class = row['class']

        img = get_img(img_src, is_train)

        xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']  # Cuadrado que encierra el objeto

        cropped_img = img[ymin:ymax, xmin:xmax]

        cropped_img = cropped_img.astype('float32') / 255.0

        resized_img = cv2.resize(cropped_img, (resize, resize))

        if len(resized_img.shape) == 3 and resized_img.shape[2] == 3:
            resized_img = cv2.cvtColor(resized_img, cv2.COLOR_RGB2GRAY)

        resized_img = resized_img.reshape(resize, resize, 1)

        processed_data.append({
            'cropped': cropped_img,
            'class': img_class,
            'original': img_src,
            'normalized': resized_img
        })

    return pd.DataFrame(processed_data)

In [25]:
df_train_processed = preprocess(df_train, resize=28, is_train=True)
df_test_processed = preprocess(df_test, resize=28, is_train=False)

In [27]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Siapkan fitur dan label
X_train = np.array([x for x in df_train_processed['normalized']])
X_test = np.array([x for x in df_test_processed['normalized']])

# Label encoding
le = LabelEncoder()
y_train = le.fit_transform(df_train_processed['class'])
y_test = le.transform(df_test_processed['class'])

# Lihat label mapping
print("Label mapping:", dict(zip(le.classes_, le.transform(le.classes_))))


Label mapping: {'Paper': 0, 'Rock': 1, 'Scissors': 2}


In [29]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation="relu", input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(3, activation='softmax'))

model.summary()

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)

# Callback ModelCheckpoint
model_checkpoint = ModelCheckpoint(
    filepath='models/best_model_CNN.h5',         
    monitor='val_loss',               
    save_best_only=True,             
    mode='min',                      
    verbose=1                       
)

# Callback EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,  
    restore_best_weights=True
)
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, model_checkpoint]
)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 11, 11, 16)        4624      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 5, 5, 16)         0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 400)               0         
                                                                 
 dense_8 (Dense)             (None, 128)              